This notebook is a WIP (work in progress) notebook to redo `qh`, 
moving it away from it's current core of `py2http` and making it (hopefully) a better basis.

The better basis we want to have:
* use wrapping and routing tools of `i2`
* use `FastAPI` as the backend

Ideally, we'd like to facade our relationship with `FastAPI`, eventually.
But given the current context (new `i2mint` devs), we will not try to hard 
(not too the point of slowing us down too much).

Before `py2http`, there was `py2api`, and before that `flask_wrap`. 
We should look there for the good ideas, so we don't have to rethink them, or redo the same mistakes.


# Refactoring a FastAPI app

We'll start with a simple `FastAPI` webservice, and gradually refactor towards 
increasingly more declarative architectures.

## Base functionality

In [2]:
from qh.scrap.fastapi_refactors.fastapi_refactor_base import (
    random_integer,
    greeter,
    backend_mall,
    get_store,
)

In [10]:
random_integer()

4

In [11]:
random_integer(40, 45)

43

In [4]:
greeter()

'Hello world!'

In [5]:
print(greeter('hi', 'bob', n=4))

hi bob!
hi bob!
hi bob!
hi bob!


In [6]:
backend_mall

{'alice': {'fruit': {'apple': 1, 'banana': 2},
  'planets': {'mercury': 10, 'venus': 20, 'earth': 30}},
 'bob': {'food': {'apple': 1, 'broccoli': 2, 'carrot': 3},
  'cars': {'ford': 10, 'toyota': 20},
  'colors': {'red': 100}}}

In [8]:
store = get_store('alice')
list(store)

['fruit', 'planets']

In [9]:
store['fruit']

{'apple': 1, 'banana': 2}

## The first a-la-mano webservice

In [ ]:
# We'll test with this:
from qh.scrap.fastapi_refactors.test_fastapi_refactor_1 import test_fastapi_refactor_app

In [21]:
from qh.scrap.fastapi_refactors.fastapi_refactor_1 import app

test_fastapi_refactor_app(app)

And now we'll factor out the routing information...

In [22]:
from qh.scrap.fastapi_refactors.fastapi_refactor_2 import app

test_fastapi_refactor_app(app)

# Scrap

## Old stuff

In [2]:
from qh.scrap.store_dispatch_1 import StoreAccess

s = StoreAccess.from_uri()
s.write('test_key', 'test_value_1111')
s.read('test_key')

'test_value_1111'

In [ ]:
from qh.scrap.store_dispatch_2 import app
from qh.scrap.store_dispatch_2 import *

@app.get("/list/{uri}")
async def list_keys(uri: str):
    store = StoreAccess.from_uri(uri)
    return store.list()


@app.get("/read/{uri}/{key}")
async def read_key(uri: str, key: str):
    try:
        store = StoreAccess.from_uri(uri)
        return store.read(key)
    except KeyError:
        raise HTTPException(status_code=404, detail="Key not found")

@app.post("/write/{uri}")
async def write_key(uri: str, body: WriteRequestBody):
    store = StoreAccess.from_uri(uri)
    store.write(body.key, body.value)

app = mk_app(
    constructor,
    constructor_arg_name='uri',
    routes={
        'list': {'route': "/list/{uri}", 'args': None, 'method': 'get'},
        'read': {'route': "/read/{uri}/{key}", 'args': ['key'], 'method': 'get'},
        'write': {'route': "/read/{uri}", 'args': ['body.key', 'body.value'], 'method': 'post'},

    }
)
   

NameError: name 'app' is not defined

Hello world!
Hello world!
Hello world!
Hello world!


In [5]:
from fastapi.testclient import TestClient

from qh.scrap.store_dispatch_2 import app

client = TestClient(app)

response = client.get("/list/test_uri")
response

<Response [422 Unprocessable Entity]>

In [ ]:
print(response.status_code)
response.json()

422


{'detail': [{'type': 'missing',
   'loc': ['query', 'args'],
   'msg': 'Field required',
   'input': None,
   'url': 'https://errors.pydantic.dev/2.5/v/missing'},
  {'type': 'missing',
   'loc': ['query', 'kwargs'],
   'msg': 'Field required',
   'input': None,
   'url': 'https://errors.pydantic.dev/2.5/v/missing'}]}

In [ ]:
from qh.scrap.simple_fastapi_dispatch import app

client = TestClient(app)

r = client.get("/poke/")
assert r.json() == 'here is a peek'

r = client.get("/foo/")


TypeError: Sig.map_arguments() got an unexpected keyword argument 'x'